In [ ]:
from elasticsearch import Elasticsearch, helpers
import json
from model_config import load_embedding_model_VN2

def upload_chunks_to_elasticsearch(json_file_path: str, index_name: str):
    """
    Đọc file JSON, tạo embedding và đưa dữ liệu lên Elasticsearch.
    """
    embeddings = load_embedding_model_VN2()

    # Khởi tạo Elasticsearch client
    es = Elasticsearch(
        hosts=["http://localhost:9200"],
        basic_auth=("elastic", "BKELASTIC"),
        verify_certs=False
    )
    json_file_path = r"D:\DATN\QA_System\casestudy\pdf\data.json"
    index_name = "my_index" 
    # Kiểm tra nếu index đã tồn tại, xóa và tạo mới
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
    es.indices.create(index=index_name, body=mapping)  # Sử dụng mapping bạn đã định nghĩa

    # Đọc file JSON
    with open(json_file_path, "r", encoding="utf-8") as f:
        json_data = json.load(f)

    # Chuyển đổi json_data thành định dạng Elasticsearch bulk
    def generate_actions(documents):
        for record in documents:
            # Tạo vector embedding từ nội dung text
            embedding = embeddings.embed_query(record["text"])

            yield {
                "_op_type": "index",
                "_index": index_name,
                "_source": {
                    "text": record["text"],
                    "metadata": record["metadata"],
                    "vector": embedding
                }
            }

    # Sử dụng helpers.bulk để insert batch dữ liệu vào Elasticsearch
    helpers.bulk(es, generate_actions(json_data))

    print(f"Đã chèn {len(json_data)} bản ghi từ file JSON vào Elasticsearch.")

In [ ]:
output_json_file = r"D:\DATN\QA_System\casestudy\pdf\data.json"
index_name = "my_index"  # Thay đổi index name nếu cần
upload_to_elasticsearch(output_json_file, index_name)

test do dai doan van ban

In [ ]:
model_embedding = load_embedding_model_VN2()
tokenizer = load_tokenizer2()



def test_text_length(text: str, tokenizer, model_embedding):
    """
    Thử nghiệm với độ dài text tăng dần và ghi lại độ dài tối đa có thể tạo embedding.

    Args:
        text: Đoạn text cần thử nghiệm.
        tokenizer: Tokenizer của model embedding.
        model_embedding: Model embedding.
    """
    words = text.split()
    max_chars = 0
    max_tokens = 0
    error_occurred = False

    for i in range(1, len(words) + 1):
        partial_text = " ".join(words[:i])
        num_chars = len(partial_text)
        num_tokens = len(tokenizer.encode(partial_text, add_special_tokens=True))

        try:
            model_embedding.embed_query(partial_text)
            max_chars = num_chars
            max_tokens = num_tokens
            print(f"Processed successfully up to {i} words, {num_chars} chars, {num_tokens} tokens.")
        except Exception as e:
            print(f"Error with {i} words ({num_chars} chars, {num_tokens} tokens): {e}")
            print(f"Problematic text: {partial_text}")
            error_occurred = True
            break

    if not error_occurred:
        print(f"Successfully processed entire text: {len(text)} chars, {len(tokenizer.encode(text, add_special_tokens=True))} tokens")
    else:
        print(f"Max processed characters: {max_chars}")
        print(f"Max processed tokens: {max_tokens}")

    return error_occurred, max_chars, max_tokens

# Thử nghiệm với document 75
text_75 = json_data[75]["text"]
print("Testing document 75:")
test_text_length(text_75, tokenizer, model_embedding)



In [ ]:
test xem co doan nao >255 token hay k

In [ ]:
from model_config import load_embedding_model_VN2 ,load_tokenizer2
import json

def check_text_length(json_file_path: str, max_length: int = 255):
    """
    Kiểm tra độ dài token của các text trong file JSON và in ra các text vượt quá giới hạn.

    Args:
        json_file_path: Đường dẫn đến file JSON.
        max_length: Độ dài token tối đa.
    """
    embeddings = load_embedding_model_VN2()
    tokenizer = load_tokenizer2() # Sử dụng tokenizer của model embedding

    with open(json_file_path, "r", encoding="utf-8") as f:
        json_data = json.load(f)

    problematic_records = []

    for i, record in enumerate(json_data):
        text = record["text"]
        tokens = tokenizer.encode(text)
        length = len(tokens)

        if length > max_length:
            problematic_records.append({
                "index": i,
                "length": length,
                "text": text[:200] + "..."  # Chỉ in ra 200 ký tự đầu tiên
            })

    if problematic_records:
        print(f"Tìm thấy {len(problematic_records)} bản ghi có độ dài vượt quá {max_length} tokens:")
        for rec in problematic_records:
            print(f"- Index: {rec['index']}, Length: {rec['length']}, Text: {rec['text']}")
    else:
        print(f"Không tìm thấy bản ghi nào có độ dài vượt quá {max_length} tokens.")

# ... (phần code khác)

def main():
    # ... (phần code xử lý dữ liệu và tạo file JSON)

    # Sau khi đã lưu dữ liệu ra file JSON
    output_json_file = r"D:\DATN\QA_System\casestudy\pdf\data.json"
    check_text_length(output_json_file) # kiểm tra độ dài của text

    # ... (phần code upload lên Elasticsearch)

if __name__ == "__main__":
    main()

chuyen finaldf sang json ma k bi loi dau ngoac don ngoac kep

In [1]:
import pandas as pd
import ast
import json

execfile = r"D:\DATN\QA_System\casestudy\pdf\finaldf0.xlsx"
df = pd.read_excel(execfile)

# Sửa lỗi định dạng JSON trong cột metadata
def fix_json_format(text):
  """
  Sửa lỗi định dạng JSON bằng cách thay thế dấu ngoặc đơn bao quanh tên thuộc tính 
  bằng dấu ngoặc kép và đánh giá chuỗi như một dictionary.
  """
  try:
      # Thay thế dấu nháy đơn bao quanh tên thuộc tính bằng dấu nháy kép
      # Tuy nhiên cách này sẽ thay thế tất cả dấu nháy đơn, bao gồm cả trong giá trị
      # text = text.replace("'", '"') 
      
      # Thay thế dấu nháy đơn bao quanh key, sử dụng ast.literal_eval để đánh giá chuỗi như một dictionary
      
      corrected_text = ast.literal_eval(text)
      
      # Chuyển đổi lại thành chuỗi JSON với định dạng chính xác
      return json.dumps(corrected_text) 
  except (ValueError, SyntaxError):
    return text
    

if isinstance(df["metadata"].iloc[0], str):
    df["metadata"] = df["metadata"].apply(fix_json_format)
    df["metadata"] = df["metadata"].apply(lambda x: json.loads(x))

# Chuyển đổi DataFrame thành danh sách các dictionary
json_data = df.to_dict(orient="records")

# Lưu danh sách các dictionary vào file JSON
output_json_file = r"D:\DATN\QA_System\casestudy\pdf\finaldf0.json"
with open(output_json_file, "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

print(f"Đã lưu dữ liệu vào file JSON: {output_json_file}")

Đã lưu dữ liệu vào file JSON: D:\DATN\QA_System\casestudy\pdf\finaldf0.json
